# Crear un corpus con Tweepy

Documentación de Tweepy: http://docs.tweepy.org/en/v3.5.0/

Lo primero necesitamos cuatro códigos que nos ofrece Twitter en la web https://apps.twitter.com. Debemos tener una cuenta de Twitter y registrar una aplicación como ejemplo.

Además debemos instalar la librería mediante la orden:

`pip install tweepy`



In [1]:
#llamamos a la librería
import tweepy
#creamos cuatro variables para poner nuestros códigos. OJO, borrarlos al subirlo a github
consumer_key=''
consumer_secret=''
access_token=''
access_token_secret=''

Debemos usar los cuatro códigos para crear un token de autenticación que nos permite acceder mediante OAuth al API de Twitter.

In [2]:
#Se crea el token de autenticación
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
#obtenemos el manejador de la api
api = tweepy.API(auth)

## Consulta directa
Una primera aproximación es usar los mecanismos de consulta directa. Para realizar una consulta usamos el método search desde el objeto API. La documentación está aquí:http://docs.tweepy.org/en/v3.5.0/api.html#API.search

La función acepta los siguientes parámetros:
- q, la consulta que queremos buscar.
- lang, el idioma de los tweets que nos interesan.
- rpp, número máximo de tweets por página, hasta un máximo de 100.
- page, número de página por donde vamos descargando la consulta. Hasta un máximo de 1500 páginas. 
- since_id, identificador del tweet desde el que queremos recuperar.

Hay otros parámetros pero no son tan interesantes. 
Devuelve un objeto del tipo SearchResult.

Veamos un ejemplo, donde pedimos 100 tweets sobre "El Ministerio del Tiempo".

In [6]:
consulta=api.search(q="Ministerio del Tiempo",lang='es',rpp=100)

In [11]:
print type(consulta),len(consulta)
for t in consulta:
    print t,'\n'

<class 'tweepy.models.SearchResults'> 15
Status(contributors=None, truncated=False, text=u"RT @MdT_TVE: |TRAILER| 'Salva el tiempo' el nuevo cap\xedtulo en Realidad Virtual de El Ministerio del Tiempo. https://t.co/YBw3a7lMFQ \xbfTe atr\u2026", is_quote_status=False, in_reply_to_status_id=None, id=917769450350370816L, favorite_count=0, _api=<tweepy.api.API object at 0x0000000006EA7E10>, author=User(follow_request_sent=False, has_extended_profile=True, profile_use_background_image=False, _json={u'follow_request_sent': False, u'has_extended_profile': True, u'profile_use_background_image': False, u'default_profile_image': False, u'id': 746476457711509504L, u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme1/bg.png', u'verified': False, u'translator_type': u'none', u'profile_text_color': u'000000', u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/837354223209709569/4tjjnr71_normal.jpg', u'profile_sidebar_fill_color': u'000000', u'entities':

Como con este método tenemos la posibilidad de generar un error basado en la tasa de descarga debemos controlar la excepción RateLimitError, luego si vamos a iterar sobre este método debemos controlarla. Además entre consulta y consulta debemos esperar unos segundos por "educación". Para esperar debemos añadir un tiempo, por ello usamos la función sleep incluida en la libería time.
De forma que nuestro método queda:

In [13]:
import time

def polite_search(q, lang,api,iteraciones=5):
    #iniciamos la lista vacia
    tweets=[]
    try:
        i=iteraciones
        while(i>0):
            #consultamos
            consulta=api.search(q="Ministerio del Tiempo",lang='es',rpp=100)
            #dormimos 30 segundos
            time.sleep(30)
            #guardamos la captura en la lista de tweets
            tweets.join(consulta)
            #decrementamos el contador
            i=i-1
        return tweets
    except RateLimitError as error:
        print "Se ha excedido el límite de Twitter."
        return tweets
    except TweepError as error:
        print "Se ha producido una excepcion. "+error
        return tweets
    

## Streaming
Para capturar los mensajes sobre un tema concreto, es interesante utilizar la versión streaming de Twitter. Esto significa que cada vez que se genere un tweet o varios relacionados con la consulta nos avise y los descargue. De forma que podemos obtener gran cantidad de tweets en tiempo real.


Está explicado en los docs: http://docs.tweepy.org/en/v3.5.0/streaming_how_to.html

Los pasos para configurar y usar la versión streaming es:
- Crear una clase que hereda de StreamListener
- Usar esa clase para crear un objeto de tipo Stream.
- Conectarse a la API de Twitter utilizando Stream. OJO, para el stream debe ser un tema que generé en tiempo real tweets. Hoy por ejemplo, el tema Cataluña está en pleno trending topic.


In [14]:
#creamos una clase que hereda a StreamListener
class MiStreamListener(tweepy.StreamListener):
    
    def on_status(self, status):
        #al recibir el tweet lo imprimimos
        print(status.text)
    
    def on_error(self, status_code):
        if status_code == 420:
            #devolver False en on_data desconecta el stream
            return False

In [21]:
#Abrimos un stream de Twitter usando nuestra clase
msl= MiStreamListener()
miStream = tweepy.Stream(auth = api.auth, listener=msl)

In [23]:
#lo usamos para seguir un tema, ojo quedará abierto ejecutando la instrucción que hemos dejado
miStream.filter(track=[u'Cataluña'])

#Cataluña: La tensión aumenta en #España a la espera del discurso de Puigdemont https://t.co/SytUhUJHaV
RT @plutonio915: El al andalus ya lo tienen en casa! https://t.co/vcy3xAleon
RT @carnecrudaradio: Pase lo que pase, que media Cataluña se quiera ir es un fracaso del gobierno de Rajoy que no lo ha evitado ni les ha c…
Por montar un debate. En el país en que las Humanidades han dejado de importar no se por qué no me sorprende q no s… https://t.co/BXeTRlnrk4
RT @jatirado: El FMI pide diálogo en la crisis de Cataluña y advierte del riesgo de contagio https://t.co/0tEKFOJR2B https://t.co/JJxA9ENtiQ
Ocho ganadores del Nobel de la Paz piden una mediación en Cataluña https://t.co/vabtCPWAHa
RT @abc_es: 🔴#ÚLTIMAHORA eDreams traslada sus filiales españolas de Cataluña a Madrid https://t.co/3qIMQII6Ia
Ay que razón tienes. https://t.co/bSlwzvbX5r
RT @elmundotoday: El ISIS recuerda que da igual que Cataluña se independice de España porque seguirá siendo parte de Al Andalus… 
RT @alvarogafu: 500 

KeyboardInterrupt: 

De esta manera, esta muy bien porque podemos mostrar los tweets, pero no nos sirve de mucho. Primero porque los estamos mostrando por pantalla. Segundo porque tenemos que casi matar el proceso. 

Modificamos la clase MiStreamListener para poder almacenar los tweets a fichero y repetimos el proceso.

In [52]:
#creamos una clase que hereda a StreamListener
class StreamWriter(tweepy.StreamListener):
    def __init__(self, file_name, *args, **kwargs):
        tweepy.StreamListener.__init__(self,*args, **kwargs)
        self.file_name = file_name
        

    def on_data(self, data):
        try:
            with open(self.file_name, 'a') as f:
                f.write(data)
        except Exception as e:
            logger.warning(e)
        return True

In [53]:
#Abrimos un stream de Twitter usando nuestra clase
msl= StreamWriter("fichero.txt")
miStream = tweepy.Stream(auth = api.auth, listener=msl)

In [54]:
#capturamos los tweets, vamos a capturar también excepciones por si acaso
try:
    miStream.filter(track=[u'Cataluña'])
except Exception as e:
    print "Excepción durante la ejecución {0} {1}".format(e.errno, e.strerror)

KeyboardInterrupt: 